In [1]:
import pandas as pd

In [2]:
#tickers = pd.read_csv('./tickers.csv', header = None)
tickers_df = pd.read_csv('./tickers.csv', dtype={'Symbol': str})
tickers_df['Symbol'] = tickers_df['Symbol'].str.strip()
tickers_df.head()

,Symbol
0,AAPL
1,NVDA
2,MSFT
3,AMZN
4,META


In [66]:
import requests
import csv
import pandas as pd

tickers_df = pd.read_csv('./tickers.csv', dtype={'Symbol': str})
tickers_df['Symbol'] = tickers_df['Symbol'].str.strip()

url = 'https://yahoo-finance166.p.rapidapi.com/api/stock/get-financial-data'

headers = {
	"x-rapidapi-key": "6fe0cabaefmsh7ef3e2799f96313p13bba4jsn8315bbcfaddb",
	"x-rapidapi-host": "yahoo-finance166.p.rapidapi.com"
}
i = 0

field_names = ['ticker','currentPrice','totalRevenue','ebitda','freeCashflow','profitMargins','revenueGrowth','debtToEquity','totalDebt','numberOfAnalystOpinions','recommendationKey','timeStamp']
stocks_info_df = pd.DataFrame(columns = field_names)

for ticker in tickers_df['Symbol']:
    # For dev purpose only
    if (i == 3):
        break
    querystring = {"region":"US","symbol":ticker}
    response = requests.get(url, headers=headers, params=querystring)
    # print(ticker,response.status_code)
    if response.status_code == 200:
        data = response.json().get('quoteSummary', []).get('result',[])#.get('0',[]).get('financialData',[])
        stock_info_dict = {}
        stock_info_dict['ticker'] = ticker
        # stock_info_dict['timestamp'] = timestamp 
        for entry in data[0]['financialData']:
            if ((entry in stocks_info_df.columns) and entry != 'recommendationKey'):
                stock_info_dict[entry] = data[0]['financialData'][entry]['raw']
            if ((entry in stocks_info_df.columns) and entry == 'recommendationKey'):
                stock_info_dict[entry] = data[0]['financialData'][entry]
        stocks_info_df = pd.concat([stocks_info_df,pd.DataFrame([stock_info_dict])], ignore_index = True)
    i += 1

In [69]:
for ticker in tickers_df['Symbol']:
    if (i == 3):
        break
    querystring = {"region":"US","symbol":ticker}
    response = requests.get(url, headers=headers, params=querystring)
    # print(ticker,response.status_code)
    if response.status_code == 200:
        data = response.json().get('quoteSummary', []).get('result',[])#.get('0',[]).get('financialData',[])
        stock_info_dict = {}
        stock_info_dict['ticker'] = ticker
        # stock_info_dict['timestamp'] = timestamp 
        for entry in data[0]['financialData']:
            if ((entry in stocks_info_df.columns) and entry != 'recommendationKey'):
                stock_info_dict[entry] = data[0]['financialData'][entry]['raw']
            if ((entry in stocks_info_df.columns) and entry == 'recommendationKey'):
                stock_info_dict[entry] = data[0]['financialData'][entry]
        stocks_info_df = pd.concat([stocks_info_df,pd.DataFrame([stock_info_dict])], ignore_index = True)
    i += 1

In [70]:
stocks_info_df

,ticker,currentPrice,totalRevenue,ebitda,freeCashflow,profitMargins,revenueGrowth,debtToEquity,totalDebt,numberOfAnalystOpinions,recommendationKey
0,AAPL,255.27,391034994688,134660997120,110846001152,0.23971,0.061,209.059,119058997248,42,buy
1,NVDA,139.67,96307003392,61184000000,33725874176,0.55041,1.224,17.221,10014999552,54,strong_buy
2,MSFT,435.25,254189993984,136551997440,61280874496,0.35608,0.160,33.657,96838000640,48,strong_buy


In [19]:
data[0]['financialData']['currentPrice']

{'raw': 254.49, 'fmt': '254.49'}

In [4]:
print(data[0]['financialData']['currentPrice']['raw'])

254.49


In [5]:
d = (data[0]['financialData'])
field_names = ['currentPrice','totalRevenue','ebitda','freeCashflow','profitMargins','revenueGrowth','debtToEquity(%)','totalDebt','numberOfAnalystOpinions','recommendationKey']
print(len(list(d['currentPrice'].keys())))

2


In [ ]:
if response.status_code == 200:
    data = response.json().get('quoteSummary', []).get('result',[])  # Extracting the 'rank' data
    csv_filename = 'daily_stock_data.csv'

    if data:
        field_names = ['rank', 'name', 'country']  # Specify required field names

        # Write data to CSV file with only specified field names
        # with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            # writer = csv.DictWriter(csvfile, fieldnames=field_names)
            # writer.writeheader()
        for entry in data:
            #writer.writerow({field: entry.get(field) for field in field_names})
            print(entry)

        print(f"Data fetched successfully and written to '{csv_filename}'")

        '''# Upload the CSV file to GCS
        bucket_name = 'bkt-ranking-data'
        #storage_client = storage.Client()
        #bucket = storage_client.bucket(bucket_name)
        destination_blob_name = f'{csv_filename}'  # The path to store in GCS

        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(csv_filename)

        print(f"File {csv_filename} uploaded to GCS bucket {bucket_name} as {destination_blob_name}")'''
    else:
        print("No data available from the API.")
else:
    print("Failed to fetch data:", response.status_code)